# Audio Moderation System: Personal Information Detection

This notebook demonstrates a mock audio moderation system that processes audio recordings to detect and flag personal information disclosure. The system uses Gemini for transcription and analysis, with PydanticAI for structured output handling.

## Setup and Dependencies

In [ ]:
import os
from pathlib import Path
from typing import List, Optional
from enum import Enum

import soundfile as sf
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent

# Load environment variables
assert load_dotenv("../.env"), "Couldn not load ../.env"
assert "GEMINI_API_KEY" in os.environ, "GEMINI_API_KEY not found in environment variables"

# Only needed on the Udacity workspace. Comment this out if running on another system.
os.environ['HF_HOME'] = '/voc/data/huggingface'
os.environ['OLLAMA_MODELS'] = '/voc/data/ollama/cache'
os.environ['HF_HUB_OFFLINE'] = '1'
os.environ['PATH'] = f"/voc/data/ollama/bin:/voc/data/ffmpeg/bin:{os.environ.get('PATH', '')}"
os.environ['LD_LIBRARY_PATH'] = f"/voc/data/ollama/lib:/voc/data/ffmpeg/lib:{os.environ.get('LD_LIBRARY_PATH', '')}"

import nest_asyncio
nest_asyncio.apply()

## Data Models and Policy Definitions

Let's assume that a company has a policy preventing people from exchanging customer information during unsecured calls or communications. This could be a possible schema for that:

In [11]:
from typing import Literal


class ViolationType(str, Enum):
    PHONE_NUMBER = "phone_number"
    EMAIL_ADDRESS = "email_address"
    SSN = "social_security_number"
    ADDRESS = "physical_address"
    CREDIT_CARD = "credit_card_number"
    DATE_OF_BIRTH = "date_of_birth"


class PolicyViolation(BaseModel):
    violation_type: ViolationType
    detected_text: str
    timestamp_start: Optional[float] = None
    context: str = Field(description="Surrounding context where violation was detected")


# TODO: Create the ModerationResult data model
# (aka output schema) for the model. Include the following
# elements:
# - transcript: str
# - violations_found: List[PolicyViolation]
# - risk_level: str (LOW, MEDIUM, or HIGH based on violations)
# - recommendations: List[str]
class ModerationResult(BaseModel):
    transcript: str = Field(
        description="Full transcript of the audio. If multiple speakers are talking, include speaker labels."
    )
    violations_found: List[PolicyViolation]
    risk_level: Literal["LOW", "MEDIUM", "HIGH"] = Field(
        description="LOW, MEDIUM, or HIGH based on violations"
    )
    recommendations: List[str]

## Audio Processing Utilities

Let's define some helper utils to prepare data for processing:

In [12]:
import librosa
import io
from pydantic_ai import Agent, BinaryContent


def audio_to_bytes(audio_array, sample_rate):
    """Convert audio array to bytes for Pydantic AI."""
    buffer = io.BytesIO()
    sf.write(buffer, audio_array, sample_rate, format='WAV')
    buffer.seek(0)
    return buffer.getvalue()

def format_audio_for_gemini(file_path: Path):
    """Format a single audio sample from the dataset."""
    audio_array, sample_rate = librosa.load(file_path, sr=None, mono=True)
    
    # Convert audio to bytes
    audio_bytes = audio_to_bytes(
        audio_array,
        sample_rate
    )
    
    # Create binary content for Pydantic AI
    # NOTE: to make this more efficient, we could upload
    # audio as mp3 instead of wav
    audio_content = BinaryContent(
        data=audio_bytes,
        media_type='audio/wav'
    )
    
    return audio_array, sample_rate, audio_content

## PydanticAI Agent Configuration

In [13]:
# TODO: Initialize the PydanticAI agent with Gemini
# and the appropriate output schema and system prompt.
# Use the 'gemini-2.5-flash-lite' model.
# The system prompt should instruct the model to:
# 1. Transcribe the audio
# 2. Analyze the transcript for personal information violations
# 3. Flag any instances of: phone numbers, email addresses, SSN, physical
#    addresses, credit card numbers, dates of birth
# 4. Assess the overall risk level and provide recommendations
moderation_agent = Agent(
    'gemini-2.5-flash-lite',
    output_type=ModerationResult,
    system_prompt="""
    You are an audio moderation system designed to detect personal information in recordings.
    
    Your task is to:
    1. Transcribe the provided audio accurately
    2. Analyze the transcript for personal information violations
    3. Flag any instances of: phone numbers, email addresses, SSN, physical addresses, credit card numbers, dates of birth
    4. Assess the overall risk level and provide recommendations
    
    Be thorough but avoid false positives. Consider context when determining if something is truly personal information.    
    """
)

## Core Moderation Functions

## Demonstration with Sample Audio

We load a sample file and apply our moderation system:

In [14]:
import textwrap

# Analyze a single audio file (modify the filename as needed)
sample_file = "../audio_moderation.mp3"

print(f"Analyzing: {sample_file}")

# TODO; Prepare audio for API. Call the format_audio_for_gemini
# function and collect the audio_array, sample_rate, and audio_content.
audio_array, sample_rate, audio_content = format_audio_for_gemini(sample_file)

# TODO: create content containing a prompt (asking the model to transcribe and
# analyze the audio) and the audio itself.
message_content = [
    "Please transcribe this audio and analyze it for personal information disclosure "
    "according to the system prompt.",
    audio_content,
]

# Process with PydanticAI agent
result = moderation_agent.run_sync(message_content)

print("\n=== ANALYSIS RESULTS ===")
print(
    f"Transcript: {textwrap.fill(result.output.transcript, width=80)}"
)
print(f"Risk Level: {result.output.risk_level}")
print(f"Violations Found: {len(result.output.violations_found)}")

if result.output.violations_found:
    print("\n=== POLICY VIOLATIONS ===")
    for i, violation in enumerate(result.output.violations_found, 1):
        print(f"{i}. {violation.violation_type.value.replace('_', ' ').title()}")
        print(f"   Detected: {violation.detected_text}")
        print(f"   Context: {violation.context}")

print("\n=== RECOMMENDATIONS ===")
for rec in result.output.recommendations:
    print(f"- {rec}")

Analyzing: ../audio_moderation.mp3

=== ANALYSIS RESULTS ===
Transcript: 00:00 Speaker 1: Hey Mike, 00:01 I just got off the phone with that difficult
customer from yesterday. 00:03 You know, Patricia Williams. 00:05 She was really
upset about the billing issue. 00:07 I had to look up all her account details to
figure out what 00:09 went wrong. 00:11 00:12 Speaker 2: Oh yeah, the one from
Portland? 00:13 What was the problem this time? 00:14 She's called like three
times this week already. 00:15 00:16 Speaker 1: Exactly, that's her. 00:17 So
apparently her automatic payment failed because her credit card ending in 7834
00:20 expired last month. 00:21 The system tried to charge the old card and she
got hit with fees. 00:24 00:27 Her new card info wasn't updated properly in our
system. 00:30 I had to manually update her profile with the new expiration date
and everything. 00:31 00:34 00:35 00:37 00:38 Speaker 2: That's frustrating.
00:39 Did you get it sorted out? 00:40 I remember she me